In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

In [2]:
df = pd.read_csv('Final_df.csv')

In [3]:
df.columns

Index(['Crop', 'Variety', 'State', 'Duration in days', 'Cost', 'Quantity',
       'Predicted_Production'],
      dtype='object')

In [4]:
X = df.drop('Predicted_Production',axis=1)
y = df['Predicted_Production']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train.shape

(220, 6)

In [7]:
X_test.shape

(56, 6)

In [8]:
X_test.head()

,Crop,Variety,State,Duration in days,Cost,Quantity
30,Paddy,CR Dhan 501 (IET 19189),Assam,152,31603.200,167.70
124,Maize,HQPM-4,Andhra Pradesh,95,26094.560,162.14
196,Indian Mustard,Pusa Mustard 26 (NPJ-113),Uttar Pradesh,132,21735.185,99.00
127,Maize,HQPM-4,Maharashtra,95,26094.560,162.14
216,Groundnut,Kadiri Harithandhra (K 1319),Maharashtra,122,17314.200,130.02


## catogerial to numerical scaling

In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [11]:
ohe = OneHotEncoder(drop='first')
scaler = StandardScaler()

In [12]:
preprocessor = ColumnTransformer(

transformers = [
    ('onehotencoder',ohe,[0,1,2]),
    ('standardization',scaler,[3,4,5])
],
remainder = 'passthrough'
)

In [13]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(drop='first'),
                                 [0, 1, 2]),
                                ('standardization', StandardScaler(),
                                 [3, 4, 5])])

In [14]:
fX_train = preprocessor.fit_transform(X_train)
fX_test = preprocessor.transform(X_test)

## Training the model

In [16]:
# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.dummy import DummyRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#from sklearn.neural_network import MLPRegressor




In [17]:

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet Regression": ElasticNet(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Hist Gradient Boosting": HistGradientBoostingRegressor(),
    "SVR": SVR(),
    "KNN Regressor": KNeighborsRegressor(),
    "MLP Regressor": MLPRegressor(),
    "Dummy Regressor (Baseline)": DummyRegressor(strategy='mean')
}

if hasattr(fX_train, "toarray"):  # Check if the data is sparse
    fX_train = fX_train.toarray()
if hasattr(fX_test, "toarray"):
    fX_test = fX_test.toarray()

# Iterate through models and fit them
for model_name, model in models.items():
    try:
        model.fit(fX_train, y_train)
        y_pred = model.predict(fX_test)

        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f'{model_name} MSE: {mse:.4f} | MAE: {mae:.4f} | R2 Score: {r2:.4f}')
    except Exception as e:
        print(f"{model_name} failed: {e}")



# Ensure the data is dense


#for model_name, model in models.items():
    # Create a pipeline
   # model.fit(fX_train,y_train)
   # y_pred = model.predict(fX_test)
  #  print(f'{model_name} MSE : {mean_squared_error(y_test,y_pred)} Score {r2_score(y_test,y_pred)}')
    

## SELECTING THE MODEL

In [19]:
dtr= DecisionTreeRegressor()
dtr.fit(fX_train,y_train)
y = dtr.predict(fX_test)



In [20]:
# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y))

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y)

# R² Score
r2 = r2_score(y_test, y)

# Print the evaluation metrics
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.2f}")

Root Mean Squared Error (RMSE): 0.00
Mean Absolute Error (MAE): 0.00
R² Score: 1.00


In [21]:
def prediction(Crop,Variety,State,Duration,Cost,Quantity):
    features = np.array([[Crop,Variety,State,Duration,Cost,Quantity]])
    transformed_features = preprocessor.transform(features)
    predicted_value = dtr.predict(transformed_features).reshape(1,-1)
    return predicted_value[0]

In [22]:
X_train

,Crop,Variety,State,Duration in days,Cost,Quantity
258,Mesta,SHRESTHA (JRM-5),Orissa,134,33198.94975,56.38
232,Sugarcane,Karan 6 (Co 0239),Haryana,360,89025.27000,131.12
33,Wheat,MPO(JW) 1215 (MPO 1215),Madhya Pradesh,120,22489.75000,116.12
157,Pearl Millet,Nandi-61 (MH-1548),Punjab,81,33198.94975,92.14
148,Pearl Millet,Nandi-65 (MH-1549),Gujarat,80,33198.94975,92.14
...,...,...,...,...,...,...
188,Indian Mustard,DRMR 601 (NRCDR 601),Punjab,144,21735.18500,99.00
71,Wheat,Pusa Prachi (HI 1563),Jharkhand,140,31428.43800,116.12
106,Barley,Pusa Losar (BH- 380),Jammu And Kashmir,190,33198.94975,73.96
270,Cotton,VBCH 2231,Gujarat,157,44211.23000,131.30


In [23]:
Crop = 'Mesta'
Variety =	"SHRESTHA (JRM-5)"
State = 'Orissa'
Duration = 134
Cost = 33198.949747
Quantity = 56.38

result = prediction(Crop,Variety,State,Duration,Cost,Quantity)


C:\Users\NAME\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\NAME\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [24]:
result

array([121.5178082])

## pickel files

In [26]:
import pickle
#pickle.dump(dtr,open('dtr.pkl','wb'))
#pickle.dump(preprocessor,open('preprocessor.pkl','wb'))

In [54]:
python -c "import sklearn; print(sklearn.__version__)"

SyntaxError: invalid syntax (110196477.py, line 1)

In [56]:
pip show scikit-learn


Name: scikit-learn
Version: 1.4.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: C:\Users\NAME\anaconda3\Lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: imbalanced-learn
Note: you may need to restart the kernel to use updated packages.


In [60]:
import sklearn
sklearn.__version__

'1.4.2'